In [ ]:
%matplotlib widget

# EEGNet
## Transfer Learning Book

In [ ]:

import sys, os
sys.path.insert(0, os.path.join(sys.path[0], '../modules'))
os.chdir('c:/Users/roryp/vscodeprojects/bci2')

## imports
from preparation import load_comp, epoch_comp, loadall_pilot, epoch_pilot, comp_channel_map3
from evaluation import EEGNet, get_fold, add_kernel_dim, onehot, test_val_rest_split, test_model
from pathlib import Path
from tensorflow.python.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split, StratifiedKFold

### Define Notebook Constants

In [ ]:
CLASSES = 3
FOLDS = 9
TRANSFER_FOLDS = 5
GOODS = ['Fz','FC3','FC1','FC2','FC4','C5','C3','C1','Cz','C2','C4','C6','CP3','CP1','CPz','CP2','CP4','P1','Pz','P2','POz']
# GOODS = ['FC3','C3','CP3','FC4','C4','CP4']
# GOODS = ['FC3','C3','CP3','Fz','Cz','POz','FC4','C4','CP4']
T_RANGE = [0.3, 2]
RESAMPLE = 128
KERNELS = 1
EPOCHS = 200
TRANSFER_EPOCHS = 300
LO_FREQ = 1.
HI_FREQ = 32.
ICA = True
WEIGHT_PATH = f"weights/competition/subject-separated/{CLASSES}class/{FOLDS}fold/channel_map3"

### Load Base Data

In [ ]:
from preparation import load_comp_array, prep_comp

subject_raws = prep_comp(load_comp_array(True), comp_channel_map3, GOODS, l_freq=LO_FREQ, h_freq=HI_FREQ)

### Create a copy of the first subject data

In [ ]:
filt_raw = subject_raws[0].copy()
filt_raw.load_data().filter(l_freq=1., h_freq=None)

### Fit ICA to Raw instance

In [ ]:
from mne.preprocessing import ICA

ica = ICA(n_components=21, random_state=97)
ica.fit(filt_raw)

Plot ICs

In [ ]:
ica.plot_sources(subject_raws[0])

In [25]:
ica.plot_properties(subject_raws[0], picks=[0,1,2])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
Not setting metadata
1345 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Not setting metadata
Not setting metadata
1345 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Not setting metadata
Not setting metadata
1345 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped


[<Figure size 400x400 with 6 Axes>,
 <Figure size 400x400 with 6 Axes>,
 <Figure size 400x400 with 6 Axes>]